# Data Warehouses and ETL on AWS


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import logging
import warnings
import boto3
import json
import pandas as pd
from rich import traceback
from pathlib import Path
from IPython.core import display as ICD

In [3]:
src_path: str = "../src"
sys.path.append(src_path)
_ = traceback.install()
logging.basicConfig(force=True)
logging.getLogger().setLevel(logging.ERROR)
warnings.filterwarnings("ignore")

In [4]:
from utils import (
    process_config,
    open_db_port,
    delete_cluster,
    delete_iam_roles,
    get_db_connection,
)
from create_dwh import main as run_create_dwh
from create_tables import main as run_create_tables
from etl import main as run_etl
from sql_queries import STAR_TABLES, get_simple_select_query

In [5]:
user_config, dwh_config = (
    process_config(Path("../_user.cfg")),
    process_config(Path("../dwh.cfg")),
)

## 1. Explore source data


### 1.1. Get S3 client

In [6]:
s3_client = boto3.resource(
    "s3",
    aws_access_key_id=user_config.get("AWS", "KEY"),
    aws_secret_access_key=user_config.get("AWS", "SECRET"),
    region_name=dwh_config.get("GENERAL", "REGION"),
)

In [7]:
udacity_bucket = s3_client.Bucket("udacity-dend")

### 1.2. Explore the song dataset

In [8]:
song_objects = [f for f in udacity_bucket.objects.limit(5).filter(Prefix="song_data")]
song_objects

[s3.ObjectSummary(bucket_name='udacity-dend', key='song_data/'),
 s3.ObjectSummary(bucket_name='udacity-dend', key='song_data/A/A/A/TRAAAAK128F9318786.json'),
 s3.ObjectSummary(bucket_name='udacity-dend', key='song_data/A/A/A/TRAAAAV128F421A322.json'),
 s3.ObjectSummary(bucket_name='udacity-dend', key='song_data/A/A/A/TRAAABD128F429CF47.json'),
 s3.ObjectSummary(bucket_name='udacity-dend', key='song_data/A/A/A/TRAAACN128F9355673.json')]

Print contents of the first few files

In [9]:
songs_preview = pd.DataFrame(
    {
        Path(s3_object.key).stem: json.loads(
            udacity_bucket.Object(s3_object.key).get()["Body"].read().decode("utf-8")
        )
        for s3_object in song_objects
        if ".json" in s3_object.key
    }
).transpose()
songs_preview

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
TRAAAAK128F9318786,ARJNIUY12298900C91,None,,None,Adelitas Way,213.9424,1,SOBLFFE12AF72AA5BA,Scream,2009
TRAAAAV128F421A322,AR73AIO1187B9AD57B,37.77916,"San Francisco, CA",-122.42005,Western Addiction,118.07302,1,SOQPWCR12A6D4FB2A3,A Poor Recipe For Civic Cohesion,2005
TRAAABD128F429CF47,ARMJAGH1187FB546F3,35.14968,"Memphis, TN",-90.04892,The Box Tops,148.03546,1,SOCIWDW12A8C13D406,Soul Deep,1969
TRAAACN128F9355673,AR9Q9YC1187FB5609B,None,New Jersey,None,Quest_ Pup_ Kevo,252.94322,1,SOFRDWL12A58A7CEF7,Hit Da Scene,0


In [10]:
songs_preview.infer_objects().dtypes

artist_id            object
artist_latitude     float64
artist_location      object
artist_longitude    float64
artist_name          object
duration            float64
num_songs             int64
song_id              object
title                object
year                  int64
dtype: object

### 1.3. Explore the log dataset

In [11]:
log_objects = [f for f in udacity_bucket.objects.limit(5).filter(Prefix="log_data")]
log_objects

[s3.ObjectSummary(bucket_name='udacity-dend', key='log_data/'),
 s3.ObjectSummary(bucket_name='udacity-dend', key='log_data/2018/11/2018-11-01-events.json'),
 s3.ObjectSummary(bucket_name='udacity-dend', key='log_data/2018/11/2018-11-02-events.json'),
 s3.ObjectSummary(bucket_name='udacity-dend', key='log_data/2018/11/2018-11-03-events.json'),
 s3.ObjectSummary(bucket_name='udacity-dend', key='log_data/2018/11/2018-11-04-events.json')]

Print contents of the first log file

In [12]:
s3_object = log_objects[1]

In [13]:
log_preview = pd.DataFrame(
    [
        json.loads(x)
        for x in (
            udacity_bucket.Object(s3_object.key)
            .get()["Body"]
            .read()
            .decode("utf-8")
            .split("\n")
        )
    ]
)
log_preview.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,None,Logged In,Walter,M,0,Frye,NaN,free,"San Francisco-Oakland-Hayward, CA",GET,Home,1.540919e+12,38,None,200,1541105830796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",39
1,None,Logged In,Kaylee,F,0,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Home,1.540345e+12,139,None,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139,You Gotta Be,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
3,None,Logged In,Kaylee,F,2,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Upgrade,1.540345e+12,139,None,200,1541106132796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
4,Mr Oizo,Logged In,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139,Flat 55,200,1541106352796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8


In [14]:
log_preview.infer_objects().dtypes

artist            object
auth              object
firstName         object
gender            object
itemInSession      int64
lastName          object
length           float64
level             object
location          object
method            object
page              object
registration     float64
sessionId          int64
song              object
status             int64
ts                 int64
userAgent         object
userId            object
dtype: object

## 2. Create Amazon Redshift cluster


In [15]:
cluster_props, redshift_client, iam_client = run_create_dwh()

Waiting for Redshift cluster to become available...
Redshift cluster is ready to be used!
ec2.SecurityGroup(id='sg-09c515e9660d60716')
An error occurred (InvalidPermission.Duplicate) when calling the AuthorizeSecurityGroupIngress operation: the specified rule "peer: 0.0.0.0/0, TCP, from port: 5439, to port: 5439, ALLOW" already exists


Reload config files after cluster creation, as they might include new fields

In [16]:
user_config, dwh_config = (
    process_config(Path("../_user.cfg")),
    process_config(Path("../dwh.cfg")),
)

## 3. Create staging and star schema tables


Create tables

In [17]:
run_create_tables()

## 4. Perform ETL


In [18]:
run_etl()

Output()

Output()

## 5. Perform some example queries for validation


### 5.1. Get DB connection

In [19]:
conn, cur = get_db_connection(dwh_config)

### 5.2. Get a preview of all star schema tables

In [20]:
query_columns = ("table_name", "column_name", "data_type")

for table_name, table_cols in STAR_TABLES.items():
    columns = [col.split(" ")[0] for col in table_cols]

    cur.execute(get_simple_select_query(table_name, ("*",), limit=5))
    print(f"Preview of '{table_name}':")
    ICD.display(pd.DataFrame(cur.fetchall(), columns=columns))
    print("\n")

Preview of 'dim_users':


,user_id,first_name,last_name,gender,level
0,52,Theodore,Smith,M,free
1,99,Ann,Banks,F,free
2,43,Jahiem,Miles,M,free
3,6,Cecilia,Owens,F,free
4,25,Jayden,Graves,M,paid




Preview of 'dim_artists':


,artist_id,name,location,latitude,longitude
0,ARSSDXP1250940EA97,Benito Kamelas,,NaN,NaN
1,ARDDQKN1187FB50651,Rednex,,NaN,NaN
2,ARZACJ71187FB42C7B,Barbecue Bob,"Walnut Grove, GA",33.74172,-83.85361
3,ARVSDB11187FB4270B,The Creation,"Middlesex, England",51.55615,-0.25969
4,AR1W00P1187B9A3A3D,Mac Lethal,Kansas City,39.10295,-94.58306




Preview of 'dim_songs':


,song_id,title,artist_id,year,duration
0,SOCBRSU12AB017B03B,City of Jerusalem,ARGLI7X1187B9930BB,2007,190
1,SOVWNAS12A8C130573,Heart Half Empty,ART09111187FB5BF48,0,295
2,SOEUYKY12A58A7A7FF,Goodbye Old Paint,ARSC7F71187FB52B7C,0,170
3,SOQUGOE12A6D4FA6FF,Awesome God,ARKC83D1187B9AB367,0,181
4,SONCXRN12A58A78503,The Silky Veils Of Ardor (LP Version),ARQXK0B1187B9ACC97,1977,241




Preview of 'dim_time':


,start_time,hour,day,week,month,year,weekday
0,2018-11-04 00:15:55,0,4,44,11,2018,0
1,2018-11-04 00:50:03,0,4,44,11,2018,0
2,2018-11-04 02:37:13,2,4,44,11,2018,0
3,2018-11-04 02:54:52,2,4,44,11,2018,0
4,2018-11-04 02:58:57,2,4,44,11,2018,0




Preview of 'fact_songplays':


,songplay_id,start_time,user_id,level,song_id,artist_id,session_id,location,user_agent
0,36,2018-11-05 18:10:41,97,paid,SOSUDUZ12AB017D1A1,Stavento,147,"Lansing-East Lansing, MI","""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5..."
1,100,2018-11-18 18:47:45,29,paid,SOTCYEO12A6D4F9DD7,Reel Big Fish,589,"Atlanta-Sandy Springs-Roswell, GA","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4..."
2,164,2018-11-28 17:06:33,26,free,SOXOURR12AB0186379,Angels Of Light & Akron/Family,991,"San Jose-Sunnyvale-Santa Clara, CA","""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5..."
3,228,2018-11-05 16:44:24,97,paid,SOBHMBE12AB0188C09,Franz Ferdinand,147,"Lansing-East Lansing, MI","""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5..."
4,292,2018-11-26 07:04:19,49,paid,SOGNKOW12A8AE47643,Rufus Wainwright,930,"San Francisco-Oakland-Hayward, CA",Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20...


### 5.3. How many records are there in each table?

In [21]:
for table_name in STAR_TABLES.keys():
    cur.execute(f"SELECT count(*) FROM {table_name}")
    print(f"{table_name} has {cur.fetchone()[0]} records.")

dim_users has 105 records.
dim_artists has 10025 records.
dim_songs has 14896 records.
dim_time has 8023 records.
fact_songplays has 1144 records.


### 5.4. Who are the top 5 users with the highest activity?

In [22]:
cur.execute(
    """
    SELECT
        sub.user_id, du.first_name, du.last_name, sub.counted
    FROM
        (
            SELECT
                fs.user_id, count(*) AS counted
            FROM
                fact_songplays fs
            JOIN
                dim_users du
            ON
                fs.user_id = du.user_id
            GROUP BY
                fs.user_id
        ) sub
    JOIN
        dim_users du ON sub.user_id = du.user_id
    ORDER BY
        sub.counted DESC, user_id
    LIMIT 5
    """
)
pd.DataFrame(cur.fetchall(), columns=("user_id", "first_name", "second_name", "count"))

,user_id,first_name,second_name,count
0,80,Tegan,Levine,147
1,49,Chloe,Cuevas,118
2,97,Kate,Harrell,84
3,24,Layla,Griffin,77
4,15,Lily,Koch,59


## 6. Close and shutdown all resources


In [ ]:
conn.close()

In [23]:
delete_cluster(redshift_client, dwh_config)
delete_iam_roles(iam_client, dwh_config)